In [33]:
#Python 3.6.3 no ambiente anaconda
#Python 3.5.5 na máquina local
from platform import python_version
print(python_version())

3.6.3



## Configurando ambiente



In [28]:
!pip install --upgrade pip setuptools wheel

     |████████████████████████████████| 1.5 MB 622 kB/s eta 0:00:01
Requirement already up-to-date: setuptools in /home/jayne/anaconda3/envs/experimento2_dissertacao/lib/python3.8/site-packages (46.1.3.post20200330)
Requirement already up-to-date: wheel in /home/jayne/anaconda3/envs/experimento2_dissertacao/lib/python3.8/site-packages (0.34.2)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2


In [29]:
!pip install Keras==1.1.0 Theano==1.0.2 numpy==1.15.1 scipy==1.3.0

  Using cached Keras-1.1.0.tar.gz (150 kB)
  Using cached Theano-1.0.2.tar.gz (2.8 MB)
  Using cached numpy-1.15.1.zip (4.5 MB)
  Using cached scipy-1.3.0.tar.gz (23.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
Failed to build Keras Theano numpy scipy pyyaml
ERROR: Could not build wheels for scipy which use PEP 517 and cannot be installed directly


In [32]:
!python --version

Python 3.8.2


## Definindo Theano como backend do keras

In [34]:
import os
os.environ["KERAS_BACKEND"] = "theano"
import keras.backend
keras.backend.set_image_dim_ordering('th')

#O formato "th" significa que os núcleos convolucionais terão a forma (profundidade, profundidade da entrada, linhas, colunas)
#O formato "tf" significa que os núcleos convolucionais terão a forma (linhas, colunas, profundidade_de_ entrada, profundidade)

## Importando LIBS

In [35]:
from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.optimizers import SGD,adam, Adagrad
from keras.regularizers import l2
import theano.tensor as T
import theano
import numpy as np
import numpy
from scipy.io import loadmat, savemat
import theano.sandbox
import urllib.request, json 

## Apagando modelos antigos
### Para refazer o upload


## Carregando modelo json e o adicionando como modelo do keras

In [36]:
DIRECTORY_GERAL = '/home/jayne/Documentos/MESTRADO/PESQUISA/AnomalyDetectionCVPR2018/'

In [37]:
json_file = open(DIRECTORY_GERAL+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
print(loaded_model_json)

# Reconstruindo um modelo a partr do arquivo de arquivo de configuração JSON e retorna uma instância do modelo.
loaded_model = model_from_json(loaded_model_json)


{
    "class_name": "Sequential",
    "keras_version": "1.1.0",
    "config": [
        {
            "class_name": "Dense",
            "config": {
                "W_constraint": null,
                "b_constraint": null,
                "name": "dense_1",
                "output_dim": 512,
                "activity_regularizer": null,
                "trainable": true,
                "init": "glorot_normal",
                "bias": true,
                "input_dtype": "float32",
                "input_dim": 4096,
                "b_regularizer": null,
                "W_regularizer": {
                    "l2": 0.0010000000474974513,
                    "name": "WeightRegularizer",
                    "l1": 0.0
                },
                "activation": "relu",
                "batch_input_shape": [
                    null,
                    4096
                ]
            }
        },
        {
            "class_name": "Dropout",
            "config": {
             


### **Iterando** sob as camadas carregadas para adicionar as camadas em uma nova rede


In [38]:
model = Sequential()

for layer in loaded_model.layers:
    model.add(layer)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 512)           2097664     dense_input_2[0][0]              
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 512)           0           dense_1[0][0]                    
                                                                   dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 32)            16416       dropout_1[0][0]                  
                                                                   dropout_1[1][0]                  
___________________________________________________________________________________________

In [39]:

for layer in model.layers:
    print(layer.trainable)

True
True
True
True
True


## Congelando camadas da  rede

In [40]:
# Congelamos a rede

for layer in model.layers[0:]:
    layer.trainable = False
     
#model.layers[2].trainable = True  
#model.layers[4].trainable = True  

# Imprima

for layer in model.layers:
    print(layer.trainable)


False
False
False
False
False


## Carregando os pesos da rede

In [41]:
# Carregamos os pesos na rede

def load_weights(model, weight_path): # Function to load the model weights
    dict2 = loadmat(weight_path)
    dict = conv_dict(dict2)
    i = 0
    for layer in model.layers:
        weights = dict[str(i)]
        layer.set_weights(weights)
        i += 1
    return model

def conv_dict(dict2):
    i = 0
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict


model2 = load_weights(model, DIRECTORY_GERAL+'weights_L1L2.mat')


# Definindo Hiperparâmetros

adagrad=Adagrad(lr=0.01, epsilon=1e-08)

model2.compile(loss='mean_squared_error', optimizer=adagrad)

In [42]:
def load_dataset_One_Video_Features(Test_Video_Path):

    VideoPath =Test_Video_Path
    f = open(VideoPath, "r")
    words = f.read().split()
    num_feat = len(words) / 4096
    # Number of features per video to be loaded. In our case num_feat=32, as we divide the video into 32 segments. Npte that
    # we have already computed C3D features for the whole video and divide the video features into 32 segments.

    count = -1;
    VideoFeatues = []
    for feat in range(0, int(num_feat)):
        feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
        count = count + 1
        if count == 0:
            VideoFeatues = feat_row1
        if count > 0:
            VideoFeatues = np.vstack((VideoFeatues, feat_row1))
    AllFeatures = VideoFeatues

    return  AllFeatures

Teste com 1 único arquivo de feature para um vídeo específico

In [43]:
FeaturePath=DIRECTORY_GERAL+'SampleVideos/Shooting008_x264.txt'
inputs = load_dataset_One_Video_Features(FeaturePath)
#inputs = np.reshape(inputs, (32, 4096))
predictions = model2.predict_on_batch(inputs)

predictions

array([[5.92888296e-02],
       [9.99995768e-01],
       [1.00000000e+00],
       [9.72513676e-01],
       [9.99530137e-01],
       [9.39119160e-01],
       [7.73924310e-03],
       [3.17960322e-01],
       [1.09492084e-02],
       [2.86138720e-06],
       [1.33183357e-06],
       [4.23348041e-07],
       [2.67143406e-07],
       [4.04276022e-07],
       [6.36074731e-07],
       [3.98849488e-06],
       [1.40022109e-06],
       [5.44635725e-07],
       [1.71988313e-06],
       [1.10032964e-07],
       [3.82445897e-08],
       [3.46304607e-09],
       [1.50118602e-08],
       [1.60915492e-09],
       [3.98882394e-09],
       [3.27964678e-09],
       [3.19895999e-09],
       [4.66307881e-09],
       [9.02528441e-10],
       [3.36979528e-10],
       [5.68824654e-10],
       [4.96184038e-10]], dtype=float32)

In [44]:
print(model2.layers[2].get_output_at(0))


Elemwise{add,no_inplace}.0


In [45]:
print(model2.input.shape)

Shape.0


In [46]:
OutFunc = keras.backend.function([model2.input], [model2.layers[0].output])
#OutFunc = keras.backend.function([model2.input], [model2.layers[2].get_output_at(0)])


out_val = OutFunc([inputs])[0]
print(out_val)

[[0.         0.04268404 0.         ... 0.05696276 0.01690462 0.04661492]
 [0.         0.         0.         ... 0.         0.         0.08978063]
 [0.         0.         0.         ... 0.         0.         0.09503081]
 ...
 [0.         0.10693794 0.         ... 0.13439566 0.11537042 0.        ]
 [0.         0.10232389 0.         ... 0.13093203 0.11200153 0.        ]
 [0.         0.10191955 0.         ... 0.13297269 0.11475941 0.        ]]


In [47]:
out_val.shape

(32, 512)

# Arquivo de features sem o cálculo da média 

In [48]:
FeaturePath=DIRECTORY_GERAL+'SampleVideos/Shooting008_x264_from_dataset.txt'
inputs = load_dataset_One_Video_Features(FeaturePath)
#inputs = np.reshape(inputs, (32, 4096))
predictions = model2.predict_on_batch(inputs)
predictions

array([[6.3002247e-01],
       [9.9998206e-01],
       [9.9999768e-01],
       [5.7959718e-01],
       [9.9961108e-01],
       [7.1074367e-01],
       [1.6969685e-03],
       [5.7201797e-01],
       [9.2288823e-04],
       [1.3315720e-06],
       [7.2199367e-07],
       [4.4238561e-07],
       [3.6952554e-07],
       [2.9708016e-07],
       [1.4987448e-06],
       [5.1049337e-06],
       [8.1995063e-07],
       [5.3264927e-07],
       [5.2073719e-06],
       [6.6955245e-08],
       [1.3930625e-08],
       [7.5535791e-09],
       [6.1032526e-09],
       [1.6890871e-09],
       [8.4789757e-09],
       [4.4935748e-09],
       [3.5126599e-09],
       [8.2814076e-09],
       [7.2168749e-10],
       [2.9396979e-10],
       [6.1429578e-10],
       [3.8280021e-10]], dtype=float32)

In [49]:
OutFunc = keras.backend.function([model2.input], [model2.layers[0].output])
#OutFunc = keras.backend.function([model2.input], [model2.layers[2].get_output_at(0)])


out_val = OutFunc([inputs])[0]
print(out_val)

[[0.         0.03073174 0.         ... 0.04886033 0.00681354 0.05338172]
 [0.         0.         0.         ... 0.         0.         0.08358815]
 [0.         0.         0.         ... 0.         0.         0.08407632]
 ...
 [0.         0.1079632  0.         ... 0.13604587 0.11634275 0.        ]
 [0.         0.10195634 0.         ... 0.13204238 0.11366762 0.        ]
 [0.         0.10201789 0.         ... 0.1339305  0.1184345  0.        ]]


In [50]:
out_val.shape

(32, 512)

In [51]:
np.save('teste1', out_val)
temp = np.load('teste1.npy')
print(temp)

[[0.         0.03073174 0.         ... 0.04886033 0.00681354 0.05338172]
 [0.         0.         0.         ... 0.         0.         0.08358815]
 [0.         0.         0.         ... 0.         0.         0.08407632]
 ...
 [0.         0.1079632  0.         ... 0.13604587 0.11634275 0.        ]
 [0.         0.10195634 0.         ... 0.13204238 0.11366762 0.        ]
 [0.         0.10201789 0.         ... 0.1339305  0.1184345  0.        ]]


In [52]:
f = open("saida.csv", "w+")

def printvalues(data, video_name, class_name):
    if isinstance(data, np.ndarray):
        f.write("x;y;value\r\n")
        for (x,y), value in np.ndenumerate(data):
            if isinstance(value, np.ndarray):
                f.write("NDArray;NDArray;NDArray\r\n")
                printvalues(value)
            else:
                f.write("{0};{1};{2}\r\n".format(str(x), str(y), str(value)))
    else:
        return 0

printvalues(out_val, "", "") 

# Calcularemos a saída da camada para dois vídeos diferentes

In [53]:
FeaturePath=DIRECTORY_GERAL+'SampleVideos/Shooting008_x264_from_dataset.txt'
inputs = load_dataset_One_Video_Features(FeaturePath)
OutFunc = keras.backend.function([model2.input], [model2.layers[0].output])
out_val = OutFunc([inputs])[0]
print(out_val)

[[0.         0.03073174 0.         ... 0.04886033 0.00681354 0.05338172]
 [0.         0.         0.         ... 0.         0.         0.08358815]
 [0.         0.         0.         ... 0.         0.         0.08407632]
 ...
 [0.         0.1079632  0.         ... 0.13604587 0.11634275 0.        ]
 [0.         0.10195634 0.         ... 0.13204238 0.11366762 0.        ]
 [0.         0.10201789 0.         ... 0.1339305  0.1184345  0.        ]]


In [54]:
FeaturePath=DIRECTORY_GERAL+'SampleVideos/Shooting033_x264.txt'
inputs = load_dataset_One_Video_Features(FeaturePath)
OutFunc = keras.backend.function([model2.input], [model2.layers[0].output])
out_val = OutFunc([inputs])[0]
print(out_val)

[[0.         0.05618518 0.         ... 0.05910342 0.0846732  0.05561975]
 [0.         0.06115815 0.         ... 0.07382973 0.08364459 0.05223005]
 [0.         0.03358655 0.         ... 0.0438396  0.05433349 0.0716617 ]
 ...
 [0.         0.04179321 0.         ... 0.03868796 0.08782981 0.0618831 ]
 [0.         0.05406076 0.         ... 0.04827485 0.08859913 0.0583579 ]
 [0.         0.04136848 0.         ... 0.03602235 0.0773719  0.06452888]]


### Teste de função

In [55]:
def save_features_layer(path, video_name, class_name):
    FeaturePath=path
    inputs = load_dataset_One_Video_Features(FeaturePath)
    OutFunc = keras.backend.function([model2.input], [model2.layers[0].output])
    out_val = OutFunc([inputs])[0]
    print(out_val)
    
    printvalues(out_val, video_name, class_name) 
    
p = DIRECTORY_GERAL+'SampleVideos/Shooting033_x264.txt'
save_features_layer(p, 0, 0)

[[0.         0.05618518 0.         ... 0.05910342 0.0846732  0.05561975]
 [0.         0.06115815 0.         ... 0.07382973 0.08364459 0.05223005]
 [0.         0.03358655 0.         ... 0.0438396  0.05433349 0.0716617 ]
 ...
 [0.         0.04179321 0.         ... 0.03868796 0.08782981 0.0618831 ]
 [0.         0.05406076 0.         ... 0.04827485 0.08859913 0.0583579 ]
 [0.         0.04136848 0.         ... 0.03602235 0.0773719  0.06452888]]


# Script para salvar a saída da camada em formato CSV

### Salvar as features extraídas da camada em um arquivo

In [56]:
import os, sys, glob, csv
import re
import time

DIRECTORY_ROOT = '/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/'
DIRECTORY_FEATURES_C3D = DIRECTORY_ROOT+'Step_2-MatLab/c3d/output_32Segments/'
DIRECTORY_FEATURES_LAYER = DIRECTORY_ROOT+'Step_3-Extracao_Ativacoes_do_Modelo/FeaturesExtractedLayerNetwork/'

#f = open(DIRECTORY_FEATURES_LAYER+"/"+"modelandotudo.csv", "w+")
csvfile = open(DIRECTORY_FEATURES_LAYER+'LayerFeatureExtractWithNovelty_test.csv', 'w+') 

def printvalues(data, video_name1, class_name1):
    video_name = video_name1
    class_name = class_name1
    if isinstance(data, np.ndarray):
        #with open(DIRECTORY_FEATURES_LAYER+'layer_feature_oficial_all.csv', 'w+') as csvfile:
        f = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        f.writerow(['video_name', 'class_name','x', 'y', 'value'])
        #f.write("video_name;video_class;x;y;value\r\n")
        for (x,y), value in np.ndenumerate(data):
            if isinstance(value, np.ndarray):
                #f.write("NDArray;NDArray;NDArray\r\n")
                printvalues(value)
            else:
                f.writerow([video_name, class_name,x, y, value])
                #f.writerow(['{0}','{1}','{2}','{3}','{4}'].format(str(video_name), str(class_name), str(x), str(y), str(value)))

               # f.write("{0};{1};{2};{3};{4}\r\n".format(str(video_name), str(class_name), str(x), str(y), str(value)))
    else:
        return 0

    
def save_features_layer(path, video_name, class_name):
    FeaturePath=path
    inputs = load_dataset_One_Video_Features(FeaturePath)
    OutFunc = keras.backend.function([model2.input], [model2.layers[0].output])
    out_val = OutFunc([inputs])[0]
    #print(out_val)
    
    printvalues(out_val, video_name, class_name) 

    
def run():
    new_directory = create_directory()
    print("List of all directories in '% s':" % DIRECTORY_FEATURES_C3D)
    with os.scandir(DIRECTORY_FEATURES_C3D) as itr: 
        for entry in itr : 
             
            if entry.is_dir() : 
                # Exclude the directory name starting with '.'   
                if not entry.name.startswith('.') :     
                    # Print Directory name   
                    class_name = entry.name
                    dire = DIRECTORY_FEATURES_C3D+class_name+'/'
                    print(dire)                    
                    files = os.listdir(dire)
                                                            
                    for video_name in files:
                        #Se o arquivo for txt
                        if video_name.endswith('.txt'):
                            path = DIRECTORY_FEATURES_C3D+class_name+'/'+video_name
                            print(path)
                            save_features_layer(path, video_name, class_name)
                            
def create_directory():
    sep = '.'
    dirName = DIRECTORY_FEATURES_LAYER
    #dirName = 'FeaturesExtractedLayerNetwork'
    #dirName = DIRECTORY+dirName

    if not os.path.exists(dirName):
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists")

    return dirName

run()

Directory  /home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_3-Extracao_Ativacoes_do_Modelo/FeaturesExtractedLayerNetwork/  already exists
List of all directories in '/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/':
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g25_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g10_c03_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g23_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g21_c04_C.txt
/home/jayne/Doc

/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g18_c02_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g06_c03_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g21_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g17_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g09_c07_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g18_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g14_c05_C.txt
/home/jayne/Documentos/MEST

/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g23_c02_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g04_c03_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g21_c07_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g22_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g07_c03_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g24_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/Punch/v_Punch_g14_c06_C.txt
/home/jayne/Documentos/MEST

/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g07_c04_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g06_c04_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g08_c02_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g25_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g16_c07_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g05_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope

/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g14_c01_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g05_c01_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g24_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g20_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g09_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g20_c02_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope

/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g19_c01_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g13_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g16_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g17_c03_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g04_c07_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope_g13_c02_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/JumpRope/v_JumpRope

/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g03_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g14_c01_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g11_c03_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g06_c05_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g01_c07_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g12_c06_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSp

/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g20_c02_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g17_c04_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g19_c04_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g20_c04_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g13_c04_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSpin/v_SalsaSpin_g03_c03_C.txt
/home/jayne/Documentos/MESTRADO/PESQUISA/Rep_Geral_Projeto_VideoVigilancia/Step_2-MatLab/c3d/output_32Segments/SalsaSp

# FIM

# Rascunhos

### Teste de escrita em arquivo

In [26]:
f = open(+"DIRECTORY_FEATURES_LAYER/modelando.csv", "w+")

def printvalues(data):
  if isinstance(data, np.ndarray):
    f.write("x;y;value\r\n")
    for (x,y), value in np.ndenumerate(data):
      if isinstance(value, np.ndarray):
        f.write("NDArray;NDArray;NDArray\r\n")
        printvalues(value)
      else:
        f.write("{0};{1};{2}\r\n".format(str(x), str(y), str(value)))
  else:
    return 0

printvalues(out_val) 

TypeError: bad operand type for unary +: 'str'

In [ ]:
savemat('outlayer2', temp)

In [ ]:
f = open("out_val_layer2.csv", "w+")

def printvalues(data):
  if isinstance(data, np.ndarray):
    f.write("x;y;value\r\n")
    for (x,y), value in np.ndenumerate(data):
      if isinstance(value, np.ndarray):
        f.write("NDArray;NDArray;NDArray\r\n")
        printvalues(value)
      else:
        f.write("{0};{1};{2}\r\n".format(str(x), str(y), str(value)))
  else:
    return 0

printvalues(out_val)     

In [ ]:
#pip install keract
from keract import get_activations
activations = get_activations(model2, inputs,[model2.layers[2].get_output_at(0)])